In [75]:
import pandas as pd
import pyodbc 
import numpy as np


In [76]:
#initial cleaning function
def clean_sheet(dataframe):
    adj=dataframe.dropna(axis=1,how='all')
    adj=dataframe.set_index(adj.columns[0]) #use stock tickers as the index
    return adj

In [77]:
cc=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\OneDrive\Post result data\{0} 2007-2017.xlsx'.format('Asia'),sheet_name='Index'))

In [97]:
df=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\OneDrive\Post result data\{0} {1}.xlsx'.\
                                     format("US","2007-2017"),sheet_name="Property",header=1))

In [100]:
df["Supersector"].unique()

array(['#N/A Review', 'Oil & Gas', 'Health Care', 'Technology',
       'Industrial Goods & Services', 'Personal & Household Goods',
       'Construction & Materials', nan, 'Retail', 'Telecommunications',
       'Financial Services', 'Utilities', 'Travel & Leisure', 'Banks',
       'Media', 'Food & Beverage', 'Basic Resources', 'Real Estate',
       'Insurance', 'Automobiles & Parts', 'Chemicals'], dtype=object)

In [143]:
def fundamental_df_clean(region,year,tabs,volume_low,size_low):
    data_dict={}

    for s in tabs:
        df=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\OneDrive\Post result data\{0} {1}.xlsx'.\
                                     format(region,year),sheet_name=s,header=1))
        data_dict[s]=df

    # select the right universe
 
    
    
    fundamental_df=data_dict["Date"].unstack(-1).to_frame()
    fundamental_df.index=fundamental_df.apply(lambda x: x.name[1],axis=1)
    fundamental_df.columns=["Date"]
    
    for i in tabs:
        if i=='Orig date' and data_dict["Orig date"].shape[1]!=data_dict["Date"].shape[1]:
            fundamental_df[i]=pd.concat([data_dict["Orig date"],data_dict["Date"].iloc[:,data_dict["Orig date"].shape[1]-\
                                                                   data_dict["Date"].shape[1]:]],axis=1).unstack(-1).values
        else:
            try:
                fundamental_df[i]=data_dict[i].unstack(-1).values
            except ValueError:
                pass
        
    fundamental_df=fundamental_df.dropna(how="all",axis=0)
    fundamental_df=fundamental_df[-np.isnat(fundamental_df.Date)]

    fundamental_df["Supersector"]=fundamental_df.apply(lambda x:data_dict["Property"].loc[x.name,"Supersector"]\
                                           if type(x.name)==str else None,axis=1)

    fundamental_df["Sector"]=fundamental_df.apply(lambda x:data_dict["Property"].loc[x.name,"Sector"]\
                                           if type(x.name)==str else None,axis=1)




    if 'Index' in data_dict.keys():
        fundamental_df["Index"]=fundamental_df.apply(lambda x: data_dict["Index"].loc[x.name].iloc[0] if type(x.name)==str\
                                                     else None,axis=1)

    fundamental_df=fundamental_df[fundamental_df.apply(lambda x: type(x.name)==str,axis=1)]

    fundamental_df=fundamental_df[(fundamental_df["Market cap"]>size_low)&(fundamental_df["Volume"]>volume_low)]
    
    fundamental_df.index.name='Ticker'
    
        
    return fundamental_df.drop_duplicates()

In [79]:
US_2019=fundamental_df_clean('US','2019',['Orig date','Date','Market cap','Volume','Property'],3,500)

In [153]:
US_2018=fundamental_df_clean('US','2018',['Orig date','Date','Market cap','Volume','Property'],3,500)

In [154]:
US_history=fundamental_df_clean('US','2007-2017',['Orig date','Date','Market cap','Volume','Property'],3,500)

In [157]:
US=US_history.append(US_2018).append(US_2019)

In [134]:
Europe_2018=fundamental_df_clean('Europe','2018',['Date','Market cap','Volume','Property',\
                                                          'Quarter'],3,500)

In [71]:
Europe_2019=fundamental_df_clean('Europe','2019',['Date','Market cap','Volume','Property',\
                                                          'Quarter'],3,500)

In [72]:
Europe_history=fundamental_df_clean('Europe','2007-2017',['Date','Market cap','Volume','Property',\
                                                          'Quarter'],3,500)

In [73]:
Europe=Europe_history.append(Europe_2018).append(Europe_2019)

In [74]:
Asia_2018=fundamental_df_clean('Asia','2018',['Date','Market cap','Volume','Property',\
                                                          'Quarter'],3,500)

In [75]:
Asia_2019=fundamental_df_clean('Asia','2019',['Date','Market cap','Volume','Property',\
                                                          'Quarter'],3,500)

In [76]:
Asia_history=fundamental_df_clean('Asia','2007-2017',['Date','Market cap','Volume','Property',\
                                                          'Quarter'],3,500)

In [77]:
Asia=Asia_history.append(Asia_2018).append(Asia_2019)

In [158]:
US.to_csv(r'C:\Users\Eric.Li\OneDrive\Post result data cache\{0}_raw_fundamental_df.csv'.format("US"))

In [78]:
Europe.to_csv('Europe.csv')

In [79]:
Asia.to_csv('Asia.csv')